## AIHub Json Preprocessing

### Development Environment

In [ ]:
%pip install kss==3.7.3

KSS Argument Error: Restart Jupyter Kernel Runtime

In [ ]:
%pip install python-mecab-ko

KSS 3.7.3 matches python-mecab-ko

In [ ]:
%pip install pandas

In [1]:
import re
import os
import kss
import ray
import json
import time
from time import sleep
from tqdm import tqdm
from mecab import MeCab
import pandas as pd
from glob import glob
from itertools import chain

In [2]:
pwd

'D:\\AIHUB'

### AIHUB 행정 문서 대상 기계독해 데이터

[Source](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=569)

#### Convert JSON File to TXT File

In [ ]:
from data_preprocessing import make_train_valid_json_txt_file_path_list
from data_preprocessing import count_number_of_txt_file_with_batch_list
from data_preprocessing import write_jsontext_to_txt_file_with_batch_list

In [13]:
json_path_list = ['AIHUB_행정 문서 대상 기계독해 데이터/Training/'+ '/**/*.json', 
                  'AIHUB_행정 문서 대상 기계독해 데이터/Validation/'+ '/**/*.json']
txt_path_list = ["exploration/machine_reading_data_for_administrative_documents_pro/AIHUB_machine_reading_data_for_administrative_documents_train_", 
                 "exploration/machine_reading_data_for_administrative_documents_pro/AIHUB_machine_reading_data_for_administrative_documents_valid_"]
corpus_name = "machine_reading_data_for_administrative_documents"

In [14]:
train_json_file_list, valid_json_file_list, train_txt_file_path_list, valid_txt_file_path_list = \
    make_train_valid_json_txt_file_path_list(json_path_list, txt_path_list, corpus_name)

The number of file: 12


In [21]:
batch_size = 10000
corpus_name = "machine_reading_data_for_administrative_documents"
the_number_of_train_txt_file, the_number_of_train_txt_file_list = count_number_of_txt_file_with_batch_list(train_json_file_list, batch_size, corpus_name)

Batch Size: 10000
The number of txt file: 42


In [22]:
source_file_by_batch_train_df = pd.read_excel('source_file_by_batch/machine_reading_data_for_administrative_documents_train.xlsx', engine='openpyxl')  
source_file_by_batch_train_df

,Unnamed: 0,File,Length of Source List,The Number of txt File,Description
0,0,AIHUB_행정 문서 대상 기계독해 데이터/Training\원천데이터\TS_tabl...,125814,13,NaN
1,1,AIHUB_행정 문서 대상 기계독해 데이터/Training\원천데이터\TS_unan...,19656,2,NaN
2,2,AIHUB_행정 문서 대상 기계독해 데이터/Training\원천데이터\TS_mult...,30170,4,NaN
3,3,AIHUB_행정 문서 대상 기계독해 데이터/Training\원천데이터\TS_span...,127864,13,NaN
4,4,AIHUB_행정 문서 대상 기계독해 데이터/Training\원천데이터\TS_text...,33744,4,NaN
5,5,AIHUB_행정 문서 대상 기계독해 데이터/Training\원천데이터\TS_span...,58148,6,NaN


In [23]:
batch_size = 10000
corpus_name = "machine_reading_data_for_administrative_documents"
write_jsontext_to_txt_file_with_batch_list(train_json_file_list, train_txt_file_path_list, batch_size, the_number_of_train_txt_file_list, corpus_name)

[Size]
The number of preprocessing corpus: 42

[Order]


100%|██████████| 42/42 [00:26<00:00,  1.61it/s]


In [24]:
batch_size = 10000
corpus_name = "machine_reading_data_for_administrative_documents"
the_number_of_valid_txt_file, the_number_of_valid_txt_file_list = count_number_of_txt_file_with_batch_list(valid_json_file_list, batch_size, corpus_name)

Batch Size: 10000
The number of txt file: 11


In [25]:
source_file_by_batch_valid_df = pd.read_excel('source_file_by_batch/machine_reading_data_for_administrative_documents_valid.xlsx', engine='openpyxl')  
source_file_by_batch_valid_df

,Unnamed: 0,File,Length of Source List,The Number of txt File,Description
0,0,AIHUB_행정 문서 대상 기계독해 데이터/Validation\원천데이터\VS_ta...,23578,3,NaN
1,1,AIHUB_행정 문서 대상 기계독해 데이터/Validation\원천데이터\VS_un...,3686,1,not subject of batch. small source list.
2,2,AIHUB_행정 문서 대상 기계독해 데이터/Validation\원천데이터\VS_mu...,5626,1,not subject of batch. small source list.
3,3,AIHUB_행정 문서 대상 기계독해 데이터/Validation\원천데이터\VS_sp...,23970,3,NaN
4,4,AIHUB_행정 문서 대상 기계독해 데이터/Validation\원천데이터\VS_te...,6308,1,not subject of batch. small source list.
5,5,AIHUB_행정 문서 대상 기계독해 데이터/Validation\원천데이터\VS_sp...,10916,2,NaN


In [26]:
batch_size = 10000
corpus_name = "machine_reading_data_for_administrative_documents"
write_jsontext_to_txt_file_with_batch_list(valid_json_file_list, valid_txt_file_path_list, batch_size, the_number_of_valid_txt_file_list, corpus_name)

[Size]
The number of preprocessing corpus: 11

[Order]


100%|██████████| 11/11 [00:06<00:00,  1.69it/s]


#### Preprocess TXT File

In [ ]:
from sentence_segmentation import preprocessing_text
from data_preprocessing import make_pro_post_txt_file_path_list
from data_preprocessing import merge_and_deduplicate_corpus_txt
from reading_data import reading_txt

In [ ]:
pro_corpus_path = "AIHUB_corpus/exploration/machine_reading_data_for_administrative_documents_pro/AIHUB_machine_reading_data_for_administrative_documents_train_" + "*.txt"
pro_post_xlsx_path = "pro_post_corpus_path/machine_reading_data_for_administrative_documents.xlsx"
pro_total_corpus_path_list, post_total_corpus_path_list = make_pro_post_txt_file_path_list(pro_corpus_path, pro_post_xlsx_path)

In [ ]:
len(pro_total_corpus_path_list)

In [ ]:
pro_coprus_file = pro_total_corpus_path_list[0]
line_length = 1
data_type = "source"

reading_txt(pro_coprus_file, line_length, data_type)

In [ ]:
pro_coprus_file = pro_total_corpus_path_list[0]
line_length = 1
data_type = "preprocessing"

reading_txt(pro_coprus_file, line_length, data_type)

In [ ]:
!python -m txt_preprocessing \
    --num_cpus 4 \
    --pro_post_xlsx_path 'pro_post_corpus_path/machine_reading_data_for_administrative_documents.xlsx' \
    --start_index 0

In [ ]:
ray.shutdown()

In [ ]:
preprocessing_corpus_path = "AIHUB_corpus/exploration/machine_reading_data_for_administrative_documents_post/AIHUB_machine_reading_data_for_administrative_documents_" +"*.txt"
merge_corpus_path = 'AIHUB_corpus/duplicate/AIHUB_machine_reading_data_for_administrative_documents.txt'
deduplicate_corpus_path = 'AIHUB_corpus/AIHUB_machine_reading_data_for_administrative_documents.txt'

In [ ]:
merge_and_deduplicate_corpus_txt(preprocessing_corpus_path, merge_corpus_path, 
                                  deduplicate_corpus_path)